In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [2]:
import os
import urllib.parse

In [3]:
import requests
import brotli
from bs4 import BeautifulSoup

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [6]:
fileout_path = "../../../data/data_wikis/2.1 - sk namuwiki scraping test/"

In [7]:
logfile_name = "_seleniumtest_log.txt"

# import sk & tw politician test data

In [8]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [9]:
sktest.shape

(9960, 24)

In [36]:
# sktest["URL_NamuWiki"]=np.nan

In [39]:
sktest[3000:3100]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
3000,문병원,文炳元,19570507,남,57,광역의원비례대표선거,20140604,Party List Legislator,울산광역시,NaN,...,등록,1,NaN,NaN,NaN,사회복지,동국대 사회과학대학원 졸업(사회복지학),(현)밝은미래복지재단 이사,(전)제18대 대선 새누리당 울산선대위 선거대책위원장,NaN
3001,문병호,文炳浩,19591227,남,56,국회의원선거,20160413,Legislator,인천광역시,부평구,...,등록,0,NaN,NaN,NaN,국회의원,서울대학교 법과대학 법학과 졸업,(현)변호사(민변),(현)국민의당 인천광역시당위원장,NaN
3002,문병화,文炳華,19620404,남,52,구·시·군의회의원선거,20140604,Legislator,경기도,하남시,...,등록,0,NaN,NaN,NaN,회사원,전신서해공업전문대학(현 호원대학교) 토목과 졸업 (2년),(현) 홍운건설 이사,(현) 하남시문화원 이사,NaN
3003,문병훈,文柄勳,19790315,남,35,구·시·군의회의원선거,20140604,Legislator,서울특별시,서초구,...,등록,1,8385.0,34.17,NaN,연구원,한양대학교 도시대학원 석사과정 졸업(도시학석사),(전)경기개발연구원 도시지역계획연구부 연구원,(현)한국산지보전협회 선임연구원,NaN
3004,문보현,文普玄,19650519,남,50,국회의원선거,20160413,Legislator,전라남도,목포시,...,등록,0,NaN,NaN,NaN,정당인,나고야대학대학원 법학박사과정수료(2000.4.1~2006.3.31.),(전)목포대학교 총학생회장,(현)정의당 정책연구위원,NaN
3005,문복란,文福蘭,19560601,여,58,구·시·군의회의원선거,20140604,Legislator,서울특별시,성동구,...,등록,1,6902.0,19.15,NaN,정당인,한국방송통신대학교 교육학과 졸업,(현) 새정치민주연합 서울특별시당 성범죄예방특별위원회 부위원장,(현) 대한적십자사 서울특별시지사 성동구후원회 사무국장,NaN
3006,문봉도,文奉道,19650505,남,49,구·시·군의회의원선거,20140604,Legislator,경상남도,의령군,...,등록,1,1293.0,33.75,NaN,의령군의회의원,한국방송통신대학교 행정학과 졸업,(현)제6대 의령군의회의원,(현)제6대 의령군의회 상·하반기 산업건설위원장,NaN
3007,문봉선,文峰仙,19620310,여,52,구·시·군의회의원선거,20140604,Legislator,경기도,과천시,...,등록,1,4713.0,26.36,NaN,정당인,동국대학교 문화예술대학원 졸업(문학 석사),(전)새누리당 과천의왕 당협 문화예술분과위원장,(현)한국문인협회 과천지부 상임 부회장,NaN
3008,문상모,文鋿模,19690202,남,45,시·도의회의원선거,20140604,Legislator,서울특별시,노원구,...,등록,1,22270.0,53.83,NaN,서울특별시의회 의원,광운대학교 상담복지정책대학원 사회복지학과 재학,(현)서울특별시의회 문화체육관광위원회 위원,(현)2018 평창동계올림픽지원 및 서울시스포츠 활성화를 위한 특별위원장,NaN
3009,문상선,文相宣,19670404,남,47,구·시·군의회의원선거,20140604,Legislator,경기도,의정부시,...,등록,0,NaN,NaN,NaN,정당인,한국방송통신대학교 경영학과 1학년 제적,(전) 민주당 국회의원 김진표 정책특보,(현) 의정부시 청소년지도협의회 사무국장,NaN


In [10]:
# percent duplicates

1 - (sktest["Name_Native"].drop_duplicates().shape[0] / sktest.shape[0])

0.12550200803212852

# scrape namuwiki with selenium

In [11]:
url_stem = "https://namu.wiki"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

In [12]:
# instantiate selenium webdriver
service = Service(executable_path=ChromeDriverManager().install())

# scrape namuwiki using selenium
for index,p in sktest.iterrows():
    
    if(pd.isnull(p.URL_NamuWiki)):
    
        # construct url - encode non-ASCII search terms as ASCII
        url= url_stem+"/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))

        # reset content & has_content_flag
        has_content_flag = True
        content=""

        try:

            # request url using selenium webdriver
            driver = webdriver.Chrome(service=service)
            driver.get(url)

            # check if page has content
            elements = driver.find_elements(By.TAG_NAME, 'p')

            for e in elements:
                if(e.text==p_error_text):
                    has_content_flag = False

            if(has_content_flag):
                content = driver.page_source

            # close webdriver
            driver.quit()

            # save content
            if(has_content_flag):

                # save output as an html file
                fout = open(fileout_path+"seleniumtest_"+str(index)+"_"+p.Name_Native+".html","w",encoding="utf-8")
                fout.write(content)
                fout.close()

                # update HasNamuWiki content field in dataframe
                #sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=url

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,url,"\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Has Content")

            else:

                # update HasNamuWiki content field in dataframe
                # sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=False

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,"False","\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Empty")
            
        except KeyboardInterrupt:
        
            print("Keyboard Interrupt")
            
            break

        except Exception as e:

            print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Error")
            print(e)





2540 김해선 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%84%A0 Number of <p> elements: 4 Has Content
2541 김해숙 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%88%99 Number of <p> elements: 4 Has Content
2542 김해순 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%88%9C Number of <p> elements: 5 Empty
2543 김해연 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%97%B0 Number of <p> elements: 5 Empty
2544 김해영 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81 Number of <p> elements: 4 Has Content
2545 김해영 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81 Number of <p> elements: 4 Has Content
2546 김해정 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%A0%95 Number of <p> elements: 5 Empty
2547 김해정 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%A0%95 Number of <p> elements: 5 Empty
2548 김해정 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%A0%95 Number of <p> elements: 5 Empty
2549 김해철 https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%B2%A0 Number of <p> elements: 5 Empty
2550 김행곤 https://namu.wiki/w/%EA%B9%80%ED%96%89%EA%B3%A4 Number of <p> elements: 5 Em

2629 김형탁 https://namu.wiki/w/%EA%B9%80%ED%98%95%ED%83%81 Number of <p> elements: 4 Has Content
2630 김형호 https://namu.wiki/w/%EA%B9%80%ED%98%95%ED%98%B8 Number of <p> elements: 4 Has Content
2631 김혜경 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EA%B2%BD Number of <p> elements: 4 Has Content
2632 김혜금 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EA%B8%88 Number of <p> elements: 5 Empty
2633 김혜란 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EB%9E%80 Number of <p> elements: 4 Has Content
2634 김혜련 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EB%A0%A8 Number of <p> elements: 4 Has Content
2635 김혜련 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EB%A0%A8 Number of <p> elements: 4 Has Content
2636 김혜미 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EB%AF%B8 Number of <p> elements: 4 Has Content
2637 김혜숙 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EC%88%99 Number of <p> elements: 4 Has Content
2638 김혜숙 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EC%88%99 Number of <p> elements: 4 Has Content
2639 김혜숙 https://namu.wiki/w/%EA%B9%80%ED%98%9C%EC%88%99

2718 김희성 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%84%B1 Number of <p> elements: 4 Has Content
2719 김희성 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%84%B1 Number of <p> elements: 4 Has Content
2720 김희수 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%88%98 Number of <p> elements: 4 Has Content
2721 김희수 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%88%98 Number of <p> elements: 4 Has Content
2722 김희수 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%88%98 Number of <p> elements: 4 Has Content
2723 김희숙 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%88%99 Number of <p> elements: 4 Has Content
2724 김희영 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%98%81 Number of <p> elements: 4 Has Content
2725 김희영 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%98%81 Number of <p> elements: 4 Has Content
2726 김희원 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%9B%90 Number of <p> elements: 4 Has Content
2727 김희정 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC%A0%95 Number of <p> elements: 4 Has Content
2728 김희정 https://namu.wiki/w/%EA%B9%80%ED%9D%AC%EC

2808 남재경 https://namu.wiki/w/%EB%82%A8%EC%9E%AC%EA%B2%BD Number of <p> elements: 5 Empty
2809 남재준 https://namu.wiki/w/%EB%82%A8%EC%9E%AC%EC%A4%80 Number of <p> elements: 4 Has Content
2810 남재홍 https://namu.wiki/w/%EB%82%A8%EC%9E%AC%ED%99%8D Number of <p> elements: 5 Empty
2811 남정만 https://namu.wiki/w/%EB%82%A8%EC%A0%95%EB%A7%8C Number of <p> elements: 5 Empty
2812 남정수 https://namu.wiki/w/%EB%82%A8%EC%A0%95%EC%88%98 Number of <p> elements: 5 Empty
2813 남조영 https://namu.wiki/w/%EB%82%A8%EC%A1%B0%EC%98%81 Number of <p> elements: 5 Empty
2814 남종섭 https://namu.wiki/w/%EB%82%A8%EC%A2%85%EC%84%AD Number of <p> elements: 5 Empty
2815 남진근 https://namu.wiki/w/%EB%82%A8%EC%A7%84%EA%B7%BC Number of <p> elements: 5 Empty
2816 남진복 https://namu.wiki/w/%EB%82%A8%EC%A7%84%EB%B3%B5 Number of <p> elements: 4 Has Content
2817 남창진 https://namu.wiki/w/%EB%82%A8%EC%B0%BD%EC%A7%84 Number of <p> elements: 5 Empty
2818 남천희 https://namu.wiki/w/%EB%82%A8%EC%B2%9C%ED%9D%AC Number of <p> elements: 5 Empty
2819 남평우 

2899 도형수 https://namu.wiki/w/%EB%8F%84%ED%98%95%EC%88%98 Number of <p> elements: 5 Empty
2900 라태원 https://namu.wiki/w/%EB%9D%BC%ED%83%9C%EC%9B%90 Number of <p> elements: 5 Empty
2901 류경숙 https://namu.wiki/w/%EB%A5%98%EA%B2%BD%EC%88%99 Number of <p> elements: 5 Empty
2902 류경완 https://namu.wiki/w/%EB%A5%98%EA%B2%BD%EC%99%84 Number of <p> elements: 5 Empty
2903 류규하 https://namu.wiki/w/%EB%A5%98%EA%B7%9C%ED%95%98 Number of <p> elements: 4 Has Content
2904 류금현 https://namu.wiki/w/%EB%A5%98%EA%B8%88%ED%98%84 Number of <p> elements: 5 Empty
2905 류동열 https://namu.wiki/w/%EB%A5%98%EB%8F%99%EC%97%B4 Number of <p> elements: 5 Empty
2906 류동철 https://namu.wiki/w/%EB%A5%98%EB%8F%99%EC%B2%A0 Number of <p> elements: 5 Empty
2907 류명기 https://namu.wiki/w/%EB%A5%98%EB%AA%85%EA%B8%B0 Number of <p> elements: 5 Empty
2908 류명선 https://namu.wiki/w/%EB%A5%98%EB%AA%85%EC%84%A0 Number of <p> elements: 4 Has Content
2909 류명열 https://namu.wiki/w/%EB%A5%98%EB%AA%85%EC%97%B4 Number of <p> elements: 5 Empty
2910 류명현 

2948 마선식 https://namu.wiki/w/%EB%A7%88%EC%84%A0%EC%8B%9D Number of <p> elements: 5 Error
Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x007A5FD3+2187219]
	Ordinal0 [0x0073E6D1+1763025]
	Ordinal0 [0x00653E78+802424]
	Ordinal0 [0x006505C8+787912]
	Ordinal0 [0x006465AD+746925]
	Ordinal0 [0x006471E2+750050]
	Ordinal0 [0x006467BA+747450]
	Ordinal0 [0x00645D3F+744767]
	Ordinal0 [0x00644C28+740392]
	Ordinal0 [0x006450FD+741629]
	Ordinal0 [0x00655644+808516]
	Ordinal0 [0x006AD66D+1169005]
	Ordinal0 [0x0069CB8C+1100684]
	Ordinal0 [0x006ACFC2+1167298]
	Ordinal0 [0x0069C9A6+1100198]
	Ordinal0 [0x00676F80+946048]
	Ordinal0 [0x00677E76+949878]
	GetHandleVerifier [0x00A490C2+2721218]
	GetHandleVerifier [0x00A3AAF0+2662384]
	GetHandleVerifier [0x0083137A+526458]
	GetHandleVerifier [0x00830416+522518]
	Ordinal0 [0x00744EAB+1789611]
	Ordinal0 [0x007497A8+1808296]
	Ordinal0 [0x00749895+1808533]
	Ordinal0 [0x007526C1+1844

In [13]:
sktest[sktest.Name_Native=="강민국"]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
60,강민국,姜旻局,19710303,남,43,시·도의회의원선거,20140604,Legislator,경상남도,진주시,...,등록,1,28248.0,75.31,NaN,정당인,경남대학교 대학원 졸업(법학박사),(전)경상남도 도지사 비서실장,(전)경상남도 정무보좌역,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD


In [14]:
# things to check

# 0. drop duplicates. some duplicate pages are different (at least size differs). why?  

# 0.1 drop duplicates (by=[Name_Native, Name_Chinese, Birthdate]) in TW dataset, before scraping

# 1. saved links are valid (no false positives); False links actually don't exist (no false negatives)

# 2. disambiguation? what about names that link to more than one person?
#
#       - disambiguation links are usually included in the page
#
#       - redirect from the page of a more popular person
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD - 강민국
#               정치인에 대한 내용은 강민국(정치인) 문서를 참고하십시오.
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD(%EC%A0%95%EC%B9%98%EC%9D%B8) - 강민국(정치인)
#
#       - redirects from a page listing links to multiple people having the same name
#               https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%84

# update sktest with selenium log from latest run

In [15]:
log_columns = ["index","Name_Native","url","dropme"]
log = pd.read_csv(fileout_path+logfile_name,header=None,encoding="utf-8",names=log_columns)
log.drop(columns=["dropme"],inplace=True)
log

,index,Name_Native,url
0,2540,김해선,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%84%A0
1,2541,김해숙,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%88%99
2,2542,김해순,False
3,2543,김해연,False
4,2544,김해영,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
5,2545,김해영,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
6,2546,김해정,False
7,2547,김해정,False
8,2548,김해정,False
9,2549,김해철,False


In [16]:
sktest.shape

(9960, 24)

In [17]:
sktest_updated = sktest.merge(log,how="left",on="Name_Native")
sktest_updated.shape

(10068, 26)

In [18]:
# update URL_NamuWiki with log results
sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"URL_NamuWiki"] = sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"url"]

# confirm that changes have been made
sktest_updated.loc[sktest_updated.url.notnull(),["URL_NamuWiki","url"]]

,URL_NamuWiki,url
2540,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%84%A0,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%84%A0
2541,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%88%99,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%88%99
2542,False,False
2543,False,False
2544,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2545,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2546,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2547,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2548,False,False
2549,False,False


In [19]:
# change "False" strings to boolean False
sktest_updated.loc[sktest_updated.URL_NamuWiki=="False","URL_NamuWiki"]=False

In [20]:
sktest_updated[sktest_updated["index"].notnull()]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki,index,url
2540,김해선,金海善,19731130,여,40,구·시·군의회의원선거,20140604,Legislator,서울특별시,성동구,...,10000.0,28.34,NaN,정당인,한양사이버대학교 사회복지학과 4학년 재학중,(전) 국회의원 홍익표 비서관(5급상당),(전) 민주당성동을지역위원회 여성위원장,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%84%A0,2540.0,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%84%A0
2541,김해숙,金海淑,19640105,여,50,구·시·군의회의원선거,20140604,Legislator,경기도,성남시분당구,...,12270.0,27.63,NaN,성남시의회의원,가천대학교 경영대학원 졸업(석사),(현)성남시의회의원,(전)주민소비자생활협동조합이사장,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%88%99,2541.0,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%88%99
2542,김해순,金海順,19590403,여,55,구·시·군의회의원선거,20140604,Legislator,전라북도,군산시,...,NaN,NaN,NaN,행복한결혼상담소 소장,한일장신대학교 기독교사회복지대학원 졸업(사회복지학석사),(전)전북신체장애인협회 군산지부 총무,(현)전도사(27년 시무중),False,2542.0,False
2543,김해연,金海淵,19660710,남,47,구·시·군의 장선거,20140604,Chief,경상남도,거제시,...,NaN,NaN,NaN,회사원,부산대학교 대학원 기계공학부 졸업(공학석사),"(전)경남도의회 제8,9대 의회의원",(현)경남미래발전연구소 소장,False,2543.0,False
2544,김해영,金海永,19641209,남,49,구·시·군의회의원선거,20140604,Legislator,충청북도,괴산군,...,1753.0,17.40,NaN,삼오측량설계사무소,충북보건과학대학교 1년 재학,(전)괴산군 체육회 전문이사,(현)괴산JC특우회 회장,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,2544.0,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2545,김해영,金海永,19641209,남,49,구·시·군의회의원선거,20140604,Legislator,충청북도,괴산군,...,1753.0,17.40,NaN,삼오측량설계사무소,충북보건과학대학교 1년 재학,(전)괴산군 체육회 전문이사,(현)괴산JC특우회 회장,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,2545.0,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2546,김해영,金海永,19770206,남,39,국회의원선거,20160413,Legislator,부산광역시,연제구,...,51867.0,51.60,NaN,변호사,부산대학교 법학과 졸업,(현)변호사,(현)더불어민주당 연제구 지역위원장,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,2544.0,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2547,김해영,金海永,19770206,남,39,국회의원선거,20160413,Legislator,부산광역시,연제구,...,51867.0,51.60,NaN,변호사,부산대학교 법학과 졸업,(현)변호사,(현)더불어민주당 연제구 지역위원장,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81,2545.0,https://namu.wiki/w/%EA%B9%80%ED%95%B4%EC%98%81
2548,김해정,金海貞,19741211,여,39,구·시·군의회의원선거,20140604,Legislator,경기도,수원시영통구,...,NaN,NaN,NaN,주부,전남대학교 법과대학 행정학과 졸업,(현)수원일하는여성회 대안교육센터장,(현)통합진보당 영통구지역위원회 부위원장,False,2546.0,False
2549,김해정,金海貞,19741211,여,39,구·시·군의회의원선거,20140604,Legislator,경기도,수원시영통구,...,NaN,NaN,NaN,주부,전남대학교 법과대학 행정학과 졸업,(현)수원일하는여성회 대안교육센터장,(현)통합진보당 영통구지역위원회 부위원장,False,2547.0,False


In [21]:
# drop log column (url)
sktest_updated.drop(columns=["index","url"],inplace=True)

In [22]:
# drop duplicate rows
sktest_updated.drop_duplicates().shape

(9960, 24)

In [23]:
# drop duplicate politicians
sktest_updated.sort_values(["Name_Native","Name_Chinese","Birthdate","ElectionDate"],inplace=True)
sktest_updated.drop_duplicates(["Name_Native","Name_Chinese","Birthdate"],keep="last",inplace=True)

In [24]:
sktest_updated.shape

(9960, 24)

In [25]:
# refresh and save updated sktest
sktest = sktest_updated
sktest.to_excel(filein_path+"sk_testset.xlsx",index=False)

In [26]:
# delete log file
os.remove(fileout_path+logfile_name)

In [33]:
# DEPRECATED - assign URL_NamuWiki link based on files already scraped - I needed this before I created a log file

# import os
# filelist = os.listdir(fileout_path)
# test = sktest
# url_stem = "https://namu.wiki"
# test[100:200]
# scraped = [f.split("_")[2].split(".")[0].strip() for f in filelist]
# for s in scraped:
#     url = url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8"))
#     # print(url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8")))
#     test.loc[test.Name_Native==s,"URL_NamuWiki"]=url
# test[100:200]